In [1]:
import os
import gc
import requests
import uuid
from io import BytesIO
import numpy as np
import pandas as pd
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

from typing import Generator, Tuple
from sklearn.neighbors import NearestNeighbors

import hashlib
import logging

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("smote_process.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [2]:
# 1. Configuración inicial
LOCAL_IMAGE_PATH = './repo_dataset'
TARGET_SIZE = (224, 224)
TARGET_SIZE_CHANNEL = (224, 224, 3)
BATCH_SIZE = 32

# Columnas de clases
LABEL_COLUMNS = ['direccion', 'fachada', 'envio', 'etiqueta', 'planilla']


#cargar csv y dividir en dev set y test set
# TRAIN
#CSV_PATH = './mobilnet-multi-label-con-planilla-all-data.csv'
CSV_PATH_DEV = './mobilnet-multi-label-train-40-60-planilla.csv'

CSV_TRAIN = CSV_PATH_DEV

In [3]:
# cargar las imagenes
def prepare_image(row, local_image_path, label_columns, target_size):
    # Preparar las etiquetas
    labels = row[label_columns].values.astype(int)
 
    try:
        # Cargar desde archivo local
        img_path = os.path.join(local_image_path, row['filename'])
        if os.path.exists(img_path):
            image = Image.open(img_path)
        elif pd.notna(row['urlAbsoluta']):    
             urlAbsoluta = row['urlAbsoluta']
             if 'http' in urlAbsoluta:
                 # Descargar la imagen desde la URL
                 response = requests.get(row['urlAbsoluta'], stream=True, timeout=10)
                 if response.status_code == 200:
                     image = Image.open(BytesIO(response.content))
                     #guardar local para el siguiente ciclo de entrenamiento/prueba
                     image.save(img_path)
             elif os.path.exists(urlAbsoluta):
                 image = Image.open(urlAbsoluta)
             else:
                 raise Exception(f'Error cargando {urlAbsoluta}, archivo no encontrado')
    
        # Convertir a RGB (en caso de que la imagen esté en otro formato, como RGBA)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Redimensionar la imagen
        image = image.resize(target_size)  # Redimensionar a 224x224 para MobileNetV3
        
        # Convertir a un array de numpy y normalizar
        image = np.array(image) / 255.0  # Normalizar
        
        return image, np.array(labels)
    except BaseException as e:
        print(f'Error en: {img_path}, Excepción: {str(e)}')
        return None

def print_class_distribution_from_csv(csv_path, label_columns):
    """
    Imprime la distribución de clases leyendo desde un archivo CSV
    
    Parámetros:
    csv_path: str - Ruta al archivo CSV
    label_columns: list - Lista de nombres de las columnas de etiquetas
    """
    # Leer solo las columnas necesarias del CSV
    df = pd.read_csv(csv_path, usecols=label_columns)
    total_samples = len(df)
    frecuencias = [0] * len(label_columns)
    
    print(f"Dataset preparado con {total_samples} imágenes")
    print(f"Distribución de clases:")
    
    for idx, col in enumerate(label_columns):
        positive_samples = df[col].sum()
        percentage = (positive_samples / total_samples) * 100
        print(f"{col}: {percentage:.2f}% ({int(positive_samples)}/{total_samples})")
        frecuencias[idx] = positive_samples
    return total_samples, frecuencias

In [4]:
def batch_loader(csv_path, local_image_path, label_columns, target_size, batch_size=32):
    """Generador de lotes de ejemplo"""
    df = pd.read_csv(csv_path)
    total_samples = len(df)
    
    for start in range(0, total_samples, batch_size):
        batch = df.iloc[start:start+batch_size]
        X_batch = []
        y_batch = []
        
        for _, row in batch.iterrows():
            result = prepare_image(row, local_image_path, label_columns, target_size)
            if result is not None:
                img_array, img_labels = result
                X_batch.append(img_array)
                y_batch.append(img_labels)
            else: 
                continue
        
        yield np.array(X_batch), np.array(y_batch)

In [5]:
class MultiLabelSMOTE:
    def __init__(self, target_samples=500, k_neighbors=5, output_dir='synthetic', batch_size=100, img_shape=TARGET_SIZE_CHANNEL):
        self.target_samples = target_samples
        self.k_neighbors = k_neighbors
        self.batch_size = batch_size
        self.output_dir = output_dir
        self.csv_path = self.csv_path = os.path.join(output_dir, 'metadata.csv')
        
        os.makedirs(output_dir, exist_ok=True)
        self._init_csv()
        
        # Estado del proceso
        self.class_stats = {
            'direccion': {'original': 0, 'synthetic': 0},
            'fachada': {'original': 0, 'synthetic': 0},
            'envio': {'original': 0, 'synthetic': 0},
            'etiqueta': {'original': 0, 'synthetic': 0},
            'planilla': {'original': 0, 'synthetic': 0}
        }
        self.existing_hashes = set()
        self._load_existing_hashes()

    def _init_csv(self) -> None:
        """Inicializa el archivo CSV de metadatos"""
        if not os.path.exists(self.csv_path):
            pd.DataFrame(columns=['filename', 'urlAbsoluta', 'direccion', 
                                'fachada', 'envio', 'etiqueta', 'planilla']).to_csv(self.csv_path, index=False)

    def _load_existing_hashes(self) -> None:
        """Carga hashes existentes de ejecuciones previas"""
        hash_file = os.path.join(self.output_dir, 'image_hashes.txt')
        try:
            if os.path.exists(hash_file):
                with open(hash_file, 'r') as f:
                    self.existing_hashes = set(f.read().splitlines())
                logger.info(f"Loaded {len(self.existing_hashes)} existing hashes")
        except Exception as e:
            logger.error(f"Error loading hashes: {str(e)}")
            raise

    def _validate_batch(self, X_batch: np.ndarray, y_batch: np.ndarray) -> None:
        """Valida el formato de los datos de entrada"""
        # Validar etiquetas binarias
        if not np.array_equal(y_batch, y_batch.astype(bool)):
            raise ValueError("Las etiquetas deben ser valores binarios (0 o 1)")
        
        # Validar rango de imágenes
        if (X_batch.dtype != np.float32 and X_batch.dtype != np.float64) or np.min(X_batch) < 0 or np.max(X_batch) > 1:
            raise ValueError("Las imágenes deben estar en formato float32 o float64 y normalizadas [0, 1]")
            
        # Validar dimensiones
        if y_batch.shape[1] != 5:
            raise ValueError("Debe haber exactamente 5 etiquetas por muestra")

    def _update_stats(self, y_batch: np.ndarray) -> None:
        """Actualiza las estadísticas de conteo"""
        for label, idx in zip(['direccion', 'fachada', 'envio', 'etiqueta', 'planilla'], range(5)):
            self.class_stats[label]['original'] += y_batch[:, idx].sum()

    def _needs_generation(self, label: str) -> bool:
        """Determina si una clase necesita más muestras"""
        print(f'Necesita {label}: {self.needs_smote[label]}')
        if not self.needs_smote[label]:
            return False
            
        total = self.class_stats[label]['original'] + self.class_stats[label]['synthetic']
        return total < self.target_samples

    def _generate_safe_samples(self, X_class: np.ndarray, y_class: np.ndarray, 
                              label: str, pbar: tqdm) -> int:
        """Genera muestras sintéticas con validaciones"""
        try:
            if len(X_class) < self.k_neighbors + 1:
                logger.warning(f"Clase {label}: Muestras insuficientes ({len(X_class)}) para SMOTE")
                return 0

            needed = self.target_samples - (self.class_stats[label]['original'] + self.class_stats[label]['synthetic'])
            if needed <= 0:
                return 0
                
            print(f'Generando para {label}')
            knn = NearestNeighbors(n_neighbors=self.k_neighbors)
            knn.fit(X_class.reshape(len(X_class), -1))
            
            generated = 0
            for _ in range(min(needed, self.batch_size)):
                i = np.random.randint(0, len(X_class))
                neighbor_idx = np.random.choice(knn.kneighbors([X_class[i].flatten()])[1][0])
                gap = np.random.uniform(0, 1)
                
                synthetic = np.clip(X_class[i] + gap * (X_class[neighbor_idx] - X_class[i]), 0, 1)
                synth_hash = hashlib.md5(synthetic.tobytes()).hexdigest()
                
                if synth_hash not in self.existing_hashes:
                    self._save_sample(synthetic, y_class[i], label, synth_hash)
                    generated += 1
                    pbar.update(1)
                    
            return generated
            
        except Exception as e:
            logger.error(f"Error generando muestras para {label}: {str(e)}")
            return 0

    def _save_sample(self, img_array: np.ndarray, y: np.ndarray, 
                    label: str, img_hash: str) -> None:
        """Guarda una muestra individual con registro robusto"""
        try:
            filename = f"synth_{label}_{img_hash[:8]}.jpg"
            filepath = os.path.abspath(os.path.join(self.output_dir, filename))
            
            # Conversión validada a uint8
            if img_array.dtype != np.uint8:
                img_array = (img_array * 255).astype(np.uint8)
                
            Image.fromarray(img_array).save(filepath)
            
            # Registrar en CSV
            pd.DataFrame([{
                'filename': filename,
                'urlAbsoluta': filepath,
                'direccion': int(y[0]),
                'fachada': int(y[1]),
                'envio': int(y[2]),
                'etiqueta': int(y[3]),
                'planilla': int(y[4])
            }]).to_csv(self.csv_path, mode='a', header=False, index=False)
            
            # Actualizar estado
            self.existing_hashes.add(img_hash)
            self.class_stats[label]['synthetic'] += 1
            
            # Registrar hash
            with open(os.path.join(self.output_dir, 'image_hashes.txt'), 'a') as f:
                f.write(f"{img_hash}\n")
                
        except Exception as e:
            logger.error(f"Error guardando muestra {filename}: {str(e)}")
            raise

    def _log_progress(self) -> None:
        """Registra el progreso actual"""
        progress = []
        for label in self.class_stats:
            total = self.class_stats[label]['original'] + self.class_stats[label]['synthetic']
            progress.append(
                f"{label}: {total}/{self.target_samples} "
                f"({min(100, total/self.target_samples*100):.1f}%)"
            )
        logger.info("Progreso | " + " | ".join(progress))

    def fit_resample(self, data_loader: Generator[Tuple[np.ndarray, np.ndarray], None, None]) -> None:
        """Ejecuta el proceso completo con seguimiento detallado"""
        total_batches = len(data_loader) if hasattr(data_loader, '__len__') else None
        progress_desc = "Procesando dataset " + (f" ({total_batches} lotes)" if total_batches else " ")
        
        try:
            with tqdm(data_loader, desc=progress_desc, unit="batch", total=total_batches) as batch_pbar:
                for batch_idx, (X_batch, y_batch) in enumerate(batch_pbar):
                    # Validar lote
                    self._validate_batch(X_batch, y_batch)
                    
                    # Actualizar estadísticas
                    self._update_stats(y_batch)
                    
                    # Procesar cada clase
                    with tqdm(total=5, desc="Clases", leave=False) as class_pbar:
                        for label in ['direccion', 'fachada', 'envio', 'etiqueta', 'planilla']:
                            if self._needs_generation(label):
                                mask = y_batch[:, list(self.class_stats.keys()).index(label)] == 1
                                X_class = X_batch[mask]
                                y_class = y_batch[mask]
                                
                                generated = self._generate_safe_samples(X_class, y_class, label, batch_pbar)
                                if generated > 0:
                                    logger.debug(f"Lote {batch_idx}: Generadas {generated} para {label}")
                                    
                            class_pbar.update(1)
                            class_pbar.refresh()
                    
                    # Liberar memoria
                    del X_batch, y_batch
                    gc.collect()
                    
                    # Reporte periódico
                    if batch_idx % 10 == 0:
                        self._log_progress()
                        
            # Reporte final
            logger.info("\nPROCESO COMPLETADO")
            self._log_progress()
            
        except Exception as e:
            logger.error(f"Error en el proceso principal: {str(e)}")
            raise
        finally:
            # Cierre seguro de recursos
            if 'f' in locals():
                f.close()
            logger.info("Limpieza finalizada")

In [6]:
# Aplicar SMOTE adaptado
print('MultiLabelSMOTE...')
# calcular la mínima cantidad de muestrar a generar con un grado de tolerancia
# Suma por columna para obtener la frecuencia de cada etiqueta
 # Leer solo las columnas necesarias del CSV
print('Distribución antes de SMOTE')
total_samples, frecuencias = print_class_distribution_from_csv(CSV_TRAIN, LABEL_COLUMNS)
# Obtener el valor máximo (la cantidad máxima de veces que aparece una etiqueta)
max_frecuencia = np.max(frecuencias)
min_frecuencia = np.min(frecuencias)
# Ver cuál etiqueta es la que más aparece
etiqueta_mas_comun = np.argmax(frecuencias) 

print(f'Frecuencia de cada etiqueta: {frecuencias}')
print(f'La etiqueta que más aparece es la {etiqueta_mas_comun} con {max_frecuencia} apariciones y la que menos aparece tiene {min_frecuencia} apariciones')
#max_frecuencia = int(max_frecuencia - (max_frecuencia * 0.5))
max_frecuencia = int(max_frecuencia - min_frecuencia)
print(f'Umbral de generación: {max_frecuencia}')

print('Generando data sintética...')
# Configurar con batch_size pequeño para baja memoria
mlsmote = MultiLabelSMOTE(
    target_samples=max_frecuencia,
    output_dir='./synthetic_data',
    batch_size=500  # Ajustar según memoria disponible
)

mlsmote.needs_smote = {
            'direccion': frecuencias[0] < max_frecuencia,
            'fachada': frecuencias[1] < max_frecuencia,
            'envio': frecuencias[2] < max_frecuencia,
            'etiqueta': frecuencias[3] < max_frecuencia,
            'planilla': frecuencias[4] < max_frecuencia,
        }

mlsmote.original_counts = {
            'direccion': frecuencias[0],
            'fachada': frecuencias[1],
            'envio': frecuencias[2],
            'etiqueta': frecuencias[3],
            'planilla': frecuencias[4]
        }

mlsmote.class_stats = {
            'direccion': {'original': frecuencias[0], 'synthetic': 0},
            'fachada': {'original': frecuencias[1], 'synthetic': 0},
            'envio': {'original': frecuencias[2], 'synthetic': 0},
            'etiqueta': {'original': frecuencias[3], 'synthetic': 0},
            'planilla': {'original': frecuencias[4], 'synthetic': 0}
        }

print(mlsmote.needs_smote)

# Ejecución
mlsmote.fit_resample(batch_loader(csv_path=CSV_TRAIN, local_image_path=LOCAL_IMAGE_PATH, label_columns=LABEL_COLUMNS, target_size=TARGET_SIZE, batch_size=500))

print('Distribución de data sintética generada con SMOTE')
print_class_distribution_from_csv('./synthetic_data/metadata.csv', label_columns=LABEL_COLUMNS)
print('MultiLabelSMOTE OK')

MultiLabelSMOTE...
Distribución antes de SMOTE
Dataset preparado con 3963 imágenes
Distribución de clases:
direccion: 16.20% (642/3963)
fachada: 17.54% (695/3963)
envio: 53.65% (2126/3963)
etiqueta: 37.50% (1486/3963)
planilla: 18.67% (740/3963)
Frecuencia de cada etiqueta: [642, 695, 2126, 1486, 740]
La etiqueta que más aparece es la 2 con 2126 apariciones y la que menos aparece tiene 642 apariciones
Umbral de generación: 1484
Generando data sintética...
{'direccion': True, 'fachada': True, 'envio': False, 'etiqueta': False, 'planilla': True}


Procesando dataset  : 0batch [00:00, ?batch/s]
ses:   0%|                                                                                    | 0/5 [00:00<?, ?it/s]

Necesita direccion: True
Generando para direccion


Procesando dataset  : 451batch [00:25, 23.86batch/s]
ses:  20%|███████████████▏                                                            | 1/5 [00:18<01:14, 18.63s/it]
Procesando dataset  : 454batch [00:25, 20.63batch/s]                                     | 1/5 [00:18<01:14, 18.63s/it]

Necesita fachada: True
Generando para fachada


...

Necesita direccion: True
Necesita fachada: True
Necesita envio: False
Necesita etiqueta: False
Necesita planilla: True



ses: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 446.69it/s]
Procesando dataset  : 8batch [02:13, 16.63s/batch]                                                                     
2025-03-11 21:29:03,052 - INFO - 
PROCESO COMPLETADO


Necesita direccion: True
Necesita fachada: True
Necesita envio: False
Necesita etiqueta: False
Necesita planilla: True


2025-03-11 21:29:03,052 - INFO - Progreso | direccion: 1976/1484 (100.0%) | fachada: 2012/1484 (100.0%) | envio: 4252/1484 (100.0%) | etiqueta: 2972/1484 (100.0%) | planilla: 2102/1484 (100.0%)
2025-03-11 21:29:03,058 - INFO - Limpieza finalizada


Distribución de data sintética generada con SMOTE
Dataset preparado con 1936 imágenes
Distribución de clases:
direccion: 54.70% (1059/1936)
fachada: 54.91% (1063/1936)
envio: 2.12% (41/1936)
etiqueta: 0.83% (16/1936)
planilla: 32.13% (622/1936)
MultiLabelSMOTE OK


In [7]:
df_final = pd.read_csv('./synthetic_data/metadata.csv')
for label in LABEL_COLUMNS:
    count = df_final[label].sum()
    print(f"Muestras para {label}: {count} (Objetivo: {mlsmote.target_samples})")

Muestras para direccion: 1059 (Objetivo: 1484)
Muestras para fachada: 1063 (Objetivo: 1484)
Muestras para envio: 41 (Objetivo: 1484)
Muestras para etiqueta: 16 (Objetivo: 1484)
Muestras para planilla: 622 (Objetivo: 1484)
